In [4]:
raw_data = sc.textFile("hour_nohead.csv")
num_data = raw_data.count()
records = raw_data.map(lambda x: x.split(",")).cache()
first = records.first()
print(first)
print(num_data)

['1', '2011-01-01', '1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']
17379


In [14]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()
print("Mapping of first categorical feature column: %s", get_mapping(records, 2))

Mapping of first categorical feature column: %s {'4': 0, '1': 1, '3': 2, '2': 3}


In [16]:
mappings = [get_mapping(records, i) for i in range(2, 10)]
print(mappings)

[{'4': 0, '1': 1, '3': 2, '2': 3}, {'0': 0, '1': 1}, {'10': 0, '4': 3, '5': 6, '3': 7, '2': 8, '7': 9, '1': 5, '12': 1, '8': 2, '9': 4, '11': 10, '6': 11}, {'10': 0, '18': 14, '17': 2, '5': 15, '13': 20, '7': 19, '0': 4, '14': 9, '22': 3, '16': 11, '11': 21, '19': 13, '15': 23, '12': 1, '3': 17, '2': 18, '1': 5, '6': 22, '4': 6, '23': 16, '9': 12, '21': 10, '20': 7, '8': 8}, {'0': 0, '1': 1}, {'4': 0, '1': 1, '5': 4, '3': 5, '2': 6, '0': 2, '6': 3}, {'0': 0, '1': 1}, {'4': 0, '1': 1, '3': 2, '2': 3}]


In [21]:
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len
print("Feature vector length for categorical features: ", cat_len)
print("Feature vector length for numerical features: ", num_len)
print("Total feature vector length: ", total_len)

Feature vector length for categorical features:  57
Feature vector length for numerical features:  4
Total feature vector length:  61


In [22]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np
def extract_features(record):
    cat_vec = np.zeros(cat_len)
    i = 0
    step = 0
    for field in records[2:9]:
        m = mappings[i]
        idx = m[field]
        cat_vec[idx + step] = 1
        i = i + 1
        step = step + len(m)
    num_vec = np.array([float(field) for field in record[10:14]])
    return np.concatenate((cate_vec, num_vec))

def extract_label(record):
    return float(record[-1])

In [27]:
data = records.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))

In [28]:
first_point = data.first()
print("Raw data: ", str(first[2:]))
print("Label: " + str(first_point.label))
print("Linear Model feature vector:\n" + str(first_point.features))
print("Linear model feature vector length: " + str(len(first_point.features)))

AttributeError: 'Exception' object has no attribute 'message'